In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/Ames_Housing_Price_Data.csv', header = [0])
pd.set_option("display.max_columns", None)

# Data Cleaning Part 1 
## Handling Null Values

In [2]:
#Dividing lot area into bins of interval size of 1000. (Max - min)/1000 = 214 bins
data['lot_bucket'] = pd.cut(data['LotArea'],214)

#Here we fill NA LotFrontage values with the mean value of the corresponding bin. We first calculate the mean lot 
#frontage (mean_lf), merge the 2 dataframes, and then fill the NA values with the corresponding mean value.
mean_lf = data.groupby('lot_bucket').agg({'LotFrontage': np.mean}, ignorena = True)
data_new = pd.merge(data, mean_lf, on = 'lot_bucket', how = 'left')
data_new.rename(columns = {'LotFrontage_x':'LotFrontage', 'LotFrontage_y':'mean_LotFrontage'}, inplace = True)
data_new['LotFrontage'].fillna(round(data_new['mean_LotFrontage'],1), inplace = True)

#Replace most N/A values with none and MasVnrType and Electrical with the mode.
categorical_list = ['Alley', 'BsmtQual', 'BsmtExposure', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
                    'PoolQC', 'Fence', 'MiscFeature', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'GarageQual',
                    'GarageCond']

numerical_list = ['MasVnrArea', 'BsmtFullBath','BsmtHalfBath','BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                  'GarageCars', 'GarageArea']

data_new['MasVnrType'].fillna(data_new['MasVnrType'].mode()[0], inplace = True)
data_new['Electrical'].fillna(data_new['Electrical'].mode()[0], inplace = True)

for el in categorical_list:
    data_new[el].fillna('None', inplace = True)

for el in numerical_list:
    data_new[el].fillna(0, inplace = True)


'''data_new['Alley'].fillna('None', inplace = True)
data_new['MasVnrType'].fillna(data_new['MasVnrType'].mode()[0], inplace = True)
data_new['MasVnrArea'].fillna(0, inplace = True)
data_new['BsmtQual'].fillna('None', inplace = True)
data_new['BsmtExposure'].fillna('None', inplace = True)
data_new['Electrical'].fillna(data_new['Electrical'].mode()[0], inplace = True)
data_new['BsmtFullBath'].fillna(0, inplace = True)
data_new['BsmtHalfBath'].fillna(0, inplace = True)
data_new['FireplaceQu'].fillna('None', inplace = True)
data_new['GarageType'].fillna('None', inplace = True)
data_new['GarageYrBlt'].fillna('None', inplace = True)
data_new['GarageFinish'].fillna('None', inplace = True)
data_new['PoolQC'].fillna('None', inplace = True)
data_new['Fence'].fillna('None', inplace = True)
data_new['MiscFeature'].fillna('None', inplace = True)
data_new['BsmtCond'].fillna('None', inplace = True)
data_new['BsmtFinType1'].fillna('None', inplace = True)
data_new['BsmtFinSF1'].fillna(0, inplace = True)
data_new['BsmtFinType2'].fillna('None', inplace = True)
data_new['BsmtFinSF2'].fillna(0, inplace = True)
data_new['BsmtUnfSF'].fillna(0, inplace = True)
data_new['TotalBsmtSF'].fillna(0, inplace = True)
data_new['GarageCars'].fillna(0, inplace = True)
data_new['GarageArea'].fillna(0, inplace = True)
data_new['GarageQual'].fillna('None', inplace = True)
data_new['GarageCond'].fillna('None', inplace = True)'''

#402.88 is the ratio of LotArea/LotFrontage for all lot areas > 20,000 to fill in N/A values that did not have
#another value in their lotfrontage bucket.
data_new['LotFrontage'].fillna(data_new['LotArea']/402.88, inplace = True)

#This one house had a garage type listed but all other garage features were NaN
data_new.at[data_new['PID'] == 910201180, 'GarageType'] = 'None'

data_new.to_csv('Ames_Housing_Price_Data_cleaned.csv')

# Data Cleaning Part 2
# Merging  Housing Price Data and Real Estate Data

In [3]:
housing_price_data = pd.read_csv('Ames_Housing_Price_Data_cleaned.csv')
real_estate_data = pd.read_csv('../data/Ames_Real_Estate_Data.csv')

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#We are dropping the rows we deemed unnecessary from real estate data

real_estate_data = real_estate_data.drop(['Tier',
'Range','ZngCdPr','ZngCdSc','ZngOLPr','ZngOLSc','ClassPr_S','ClassSc_S','Legal_Pr','SchD_S',
'TxD_S','MA_Ownr1','MA_Ownr2','MA_Line1','MA_Line2','MA_City','MA_State','MA_Zip1','MA_Zip2','Rcrd_Yr','Rcrd_Mo','Inst1_No','Inst1_Yr',
'Inst1_Mo','Inst1TPr','LndAc_S','X1TPr_D','X1TSc_D','X2TPr_D','X2TSc_D','X1TPr_S','X1TSc_S',
'X2TPr_S','X2TSc_S','X2TSc_S'],axis = 1)

In [5]:
#We are merging the two datasets by PID (in housing price data) 
#and MapRefNo (real estate data)

housing_price_data = housing_price_data.merge(real_estate_data, how = 'left',\
                                              left_on= 'PID', right_on='MapRefNo')

In [6]:
# Now we are going to merge the housing_price dataset with the geo_location dataset
geo_data = pd.read_csv('../charles/data/geo/neighborhood_location_data.csv')

housing_price_data = housing_price_data.merge(geo_data, how = 'left',\
                         left_on ='Neighborhood_x', right_on = 'Short_hand')

In [7]:
housing_price_data

,Unnamed: 0,Unnamed: 0.1,PID,GrLivArea,SalePrice_x,MSSubClass,MSZoning,LotFrontage,LotArea_x,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood_x,Condition1,Condition2,BldgType_x,HouseStyle_x,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl_x,Exterior1st,Exterior2nd,MasVnrType_x,MasVnrArea,ExterQual,ExterCond,Foundation_x,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating_x,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd_x,Functional,Fireplaces_x,FireplaceQu,GarageType_x,GarageYrBlt,GarageFinish,GarageCars,GarageArea_x,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea_x,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType_x,SaleCondition,lot_bucket,mean_LotFrontage,MapRefNo,GeoRefNo,Prop_Addr,ImpAc_S,OthAc_S,TtlVal_AsrYr,ValType,LndAcX1S,ImpAcX1S,ImpAcX2S,HSTtl_D,MilVal_D,HSTtl_S,MilVal_S,AcreX_S1,AcreGr,AcreNt_S,Neighborhood_y,LotArea_y,ParType,BldgNo_S,DwlgNo_S,BldgType_y,YrBuilt,HouseStyle_y,Foundation_y,RoofMatl_y,Ext1,Ext2,MasVnrType_y,Heating_y,Central Air,GLA,TtlBsmtSF,TotRmsAbvGrd_y,Fireplaces_y,PoolArea_y,GarageType_y,GarYrBlt,Cars,GarageArea_y,YrSold_YYYY,MoSold_MM,SalePrice_y,SaleType_y,SaleCond,ParclRel,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,Short_hand,Full_Name,lat,long,distToUni
0,0,1,909176150,856,126000,30,RL,64.9,7890,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,None,None,None,0,3,2010,WD,Normal,"(7298.458, 8298.201]",64.939130,909176150.0,909176150.0,436 HAYWARD AVE,0.0,111700.0,149000.0,F,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,S&W ISU,7890.0,Dwg&Lot,0.0,1.0,1FamDet,1939.0,1-Story,C'Block,CompShg,Wd Sdng,Wd Sdng,None,GasFWA,Yes,856.0,856.0,4.0,1.0,0.0,Detachd,1939.0,2.0,399.0,2017.0,2.0,155000.0,WRDConv,Normal,Deed,436,NaN,HAYWARD,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,SWISU,South & West of Iowa State University,42.017899,-93.651731,0.66
1,1,2,905476230,1049,139500,120,RL,42.0,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,"(3299.486, 4299.229]",46.154930,905476230.0,905476230.0,3416 WEST ST,0.0,130600.0,174100.0,F,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Edwards,4235.0,Dwg&Lot,0.0,1.0,Twnhs-E,1984.0,1-Story,C'Block,CompShg,HdBoard,HdBoard,BrkFace,GasFWA,Yes,1049.0,1049.0,5.0,0.0,0.0,Attachd,1984.0,1.0,266.0,2020.0,1.0,174400.0,WRDConv,Normal,Deed,3416,NaN,WEST,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,Edwards,Edwards,42.023035,-93.673386,1.39
2,1,2,905476230,1049,139500,120,RL,42.0,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,"(3299.486, 4299.229]",46.154930,905476230.0,905476230.0,3416 WEST ST,0.0,130600.0,174100.0,F,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Edwards,4235.0,Dwg&Lot,0.0,1.0,Twnhs-E,1984.0,1-Story,C'Block,CompShg,HdBoard,HdBoard,BrkFace,GasFWA,Yes,1049.0,1049.0,5.0,0.0,0.0,Attachd,1984.0,1.0,266.0,2019.0,3.0,170000.0,WRDConv,Normal,Deed,3416,NaN,WEST,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,Edwards,Edwards,42.023035,-93.673386,1.39
3,2,3,911128020,1001,124900,30,C (all),60.0,6060,Pave,None,Reg,Lvl,

In [8]:
housing_price_data.to_csv('Ames_Housing_Price_Data_cleaned_2.csv')